In [ ]:
#//////////////////
#  smooth_lead ///
#////////////////
#---------------------------------------------------------------------
# Read in lat/lon array and return smoothed lat/lon arrays
#---------------------------------------------------------------------
#///////////////////////
#  make_SpacedArray ///
#/////////////////////
#---------------------------------------------------------------------
# make_SpacedArray evenly spaces input lead array
#---------------------------------------------------------------------
#/////////////////////
#  add_ExtraSteps ///
#///////////////////
#---------------------------------------------------------------------
# add_ExtraSteps adds extra steps between lead array
#---------------------------------------------------------------------
#/////////////////////
#  calculate_beta ///
#///////////////////
#---------------------------------------------------------------------
# Read in lat/lon array, calculate angles between coords 
#---------------------------------------------------------------------

#////////////////////////////////
#  linear_interp_alongcoords ///
#//////////////////////////////
#---------------------------------------------------------------------
# given final coordinates and data values along intitial coordinates,
# linearly interpolate data along final coordinates
#---------------------------------------------------------------------

#////////////////////////////
#  calculate_lead_angles ///
#//////////////////////////
#---------------------------------------------------------------------
# given lead coordinates, return lead angles, lead spacing, 
# and accumulated ditance along lead
#---------------------------------------------------------------------

#///////////////////////
#  find_mean_lead  ///
#/////////////////////
#---------------------------------------------------------------------
# given list of lead coordinate lists, find mean lead position
#---------------------------------------------------------------------

In [ ]:
#//////////////////
#  smooth_lead ///
#////////////////
#---------------------------------------------------------------------
# Read in lat/lon array and return smoothed lat/lon arrays
#---------------------------------------------------------------------
# INPUTS:  lead_coords -->  lat,lon array (N x 2)
#          num_params  -->  # of max exponent in function (1, 2 or 3)
#          chunk_size  -->  number of coordinates per chunk that you fit
#          new_chunk   -->  new chunk size of evenly spaced lat values
# OUTPUT:  new_coords  -->  new array of latitudes and longitudes (M x 2)
#
# DEPENDENCIES:
import numpy as np
#pip install geopy
import geopy.distance
from geopy.distance import geodesic
# pip install metpy
import metpy
from metpy import interpolate
import cartopy.crs as ccrs
from matplotlib import pyplot as plt
import scipy
from scipy.optimize import curve_fit
#-----------------------------------------------------------------------
def smooth_lead(lead_coords, num_params, chunk_size, new_chunk):
    
    # start empty lat lon arrays
    TOTALLATS = np.array([])
    TOTALLONS = np.array([])

    # define polynomial function with specified number of parameters
    if num_params == 1:
        def func(x, a, b):
            return a + b*x
    elif num_params == 2:
        def func(x, a, b, c):
            return a + b*x + c*(x**2)
    elif num_params == 3:
        def func(x, a, b, c, d):
            return a + b*x + c*(x**2) + d*(x**3)


    # create list of evenly spaced chunks along lead coordinate list
    steps = np.arange(0, lead_coords.shape[0], chunk_size)
    
    
    # index for counting loop iterations
    jj = 0
    
    for ii in steps:

        # find indices of beginning and end of chunk
        if jj == len(steps)-1:
            end = lead_coords.shape[0]
        else: end = steps[jj+1]

        # find function fit for raw lat, lon data along chunk
        popt, pcov = curve_fit(func, lead_coords[ii:end, 0], lead_coords[ii:end, 1]) 

        # create evenly space list of latitude values along chunk
        spacedlats = np.linspace(lead_coords[ii, 0], lead_coords[end-1, 0], new_chunk)
    
        # use fit function to create smooth, evenly spaced lons list
        if num_params == 1:
            spacedlons = func(spacedlats, popt[0], popt[1])
        elif num_params == 2:
            spacedlons = func(spacedlats, popt[0], popt[1],  popt[2])
        elif num_params == 3:
            spacedlons = func(spacedlats, popt[0], popt[1],  popt[2],  popt[3])

        # add to updated lat,lon list
        TOTALLATS = np.append(TOTALLATS, spacedlats)
        TOTALLONS = np.append(TOTALLONS, spacedlons)
    
        # next loop iteration
        jj = jj+1


    # combine arrays into Mx2 lat,lon array
    new_coords = np.stack((TOTALLATS,TOTALLONS), axis = 1)
        
    return new_coords

In [ ]:
#///////////////////////
#  make_SpacedArray ///
#/////////////////////
#---------------------------------------------------------------------
# make_SpacedArray evenly spaces input lead array
#---------------------------------------------------------------------
# INPUTS: lead    -> array of lead coordinates 
#                    (Nx2 with [Lat, Lon])
#         step    -> desired geodesic step size (km) (default: 10)
#                    (arc distance between coordinates) 
#         error   -> max error size on step (km) (default: 1)
#                    (error will usually be l/2 max)
#         PROJ    ->  PyProj Coordinate Reference System to use for the output
#
# OUTPUTS: LatArray, LonArray (smoothed lead coordinates)
#
# DEPENDENCIES:
# import numpy as np
# #pip install geopy
# import geopy.distance
# from geopy.distance import geodesic
# # pip install metpy
# import metpy
# from metpy import interpolate
# import cartopy.crs as ccrs
# from matplotlib import pyplot as plt
from pyproj import CRS
#---------------------------------------------------------------------

def make_SpacedArray(lead, step=10, error=1, PROJ = CRS.from_epsg(4326), show_plot = False):
    
    # create empty arrays to fill with desired coordinates
    LatArray = np.array([lead[0,0]])
    if lead[0,1] < 0:
        LonArray = np.array([lead[0,1]+360])
    else:
        LonArray = np.array([lead[0,1]])
        
        
    # index for moving along lead coordinates
    ii=0

    while ii < lead.shape[0]:    
        # break the loop once it reaches end of the lead
        # (break if the distance between given point and last point is less than step)
        if geodesic((lead[ii,0],lead[ii,1]), (lead[-1,0],lead[-1,1])).km <= step:
            break

        # begin to loop through all points(jj) after point(ii)
        for jj in range(ii,lead.shape[0]):
            # calculate distance between point(ii) and point(jj)
            ds = geodesic((lead[ii,0],lead[ii,1]), (lead[jj,0],lead[jj,1])).km
            # if ds (point(ii) - point(jj)) exceeds chosen step size
            if ds > step:

                # break geodesic between ii and jj into NumSteps, specified above
                NumSteps=round(ds/error)
                # calculate index of waypoint nearest to specified step size
                NewIndex = round(step/error)
                # create geodesic line and extract coordinate closest to step
                NewLoc = metpy.interpolate.geodesic(PROJ, (lead[ii,0],lead[ii,1]), (lead[jj,0],lead[jj,1]), NumSteps+1)[NewIndex]
                # convert to only positive longitude values
                if NewLoc[0]<0:
                    NewLoc[0] = NewLoc[0]+360
                # add new coordinate to list
                LatArray = np.append(LatArray, NewLoc[1])
                LonArray = np.append(LonArray, NewLoc[0])

                break

        # update new location in array as starting point for next iteration        
        lead[jj,0] =  NewLoc[1]
        lead[jj,1] =  NewLoc[0]
        # iterate again, starting from point(jj)
        ii = jj
    
    # create array for arcdistance between coordinates
    dsarray = np.array([])
    for ii in range(0, LatArray.shape[0]-1):
        ds = geodesic((LatArray[ii],LonArray[ii]),(LatArray[ii+1],LonArray[ii+1])).km
        dsarray = np.append(dsarray, ds)

    if show_plot == True:
        plt.plot(range(0,len(dsarray)), dsarray)
        plt.ylabel('Arcdistance')
        plt.xlabel('Site Index')
        plt.ylim(np.mean(dsarray)-2*error,np.mean(dsarray)+2*error)
    
    return LatArray, LonArray

In [5]:
#/////////////////////
#  add_ExtraSteps ///
#///////////////////
#---------------------------------------------------------------------
# add_ExtraSteps adds extra steps between lead array
#---------------------------------------------------------------------
# INPUTS: lead        -> array of lead coordinates 
#                        (Nx2 with [Lat, Lon])
#         ExtraSteps  -> number of extra steps between points
#         PROJ        ->  PyProj Coordinate Reference System to use for the output
#
# OUTPUTS: LatArray, LonArray (updated lead coordinates)
#
# DEPENDENCIES:
# import numpy as np
# # pip install metpy
# import metpy
# from metpy import interpolate
# import cartopy.crs as ccrs
# from pyproj import CRS
#---------------------------------------------------------------------

def add_ExtraSteps(lead, ExtraSteps, PROJ = CRS.from_epsg(4326)):
    
    # create arrays to fill with desired coordinates, starting with initial coordinate
    LatArray = np.array([lead[0, 0]])
    LonArray = np.array([lead[0, 1]])

   # index for moving along lead coordinates
    ii=0
    while ii < lead.shape[0]-1:    

        # create geodesic line and extract coordinate closest to step
        NewLoc = metpy.interpolate.geodesic(PROJ, (lead[ii,0],lead[ii,1]), (lead[ii+1,0],lead[ii+1,1]), 2+ExtraSteps)[1:]

        # grab new coordinates
        if len(np.shape(NewLoc)) == 1:
            NewLons =  NewLoc[0]   
            NewLats =  NewLoc[1] 
            # convert to only positive longitude values
            if NewLons<0:
                NewLons = NewLons+360
        else:
            NewLons =  NewLoc[:,0]   
            NewLats =  NewLoc[:,1] 
            # convert to only positive longitude values
            for jj in range(len(NewLons)):
                if NewLons[jj]<0:
                    NewLons[jj] = NewLons[jj]+360

        # add new coordinate to array
        LatArray = np.append(LatArray, NewLats)
        LonArray = np.append(LonArray, NewLons)

        ii = ii+1
        
    return LatArray, LonArray


In [ ]:
#/////////////////////
#  calculate_beta ///
#///////////////////
#---------------------------------------------------------------------
# Read in lat/lon array, calculate angles between coords 
#---------------------------------------------------------------------
# INPUTS:  lead_coords -->  lat,lon array (N x 2)
#          SiteIndex   -->  index of coordinate where beta is calculated
#          step_ii     -->  step size (in # indices)
#          step_ii     -->  step size (in # indices)
# OUTPUT:  beta_a,beta_d   --> angles found in asc/des directions
#          ds_a, ds_d      --> arcdistances used in asc/des directions
#          Beta            --> orientation of lead_coords at SiteIndex
#
# take in lat/lon array. For a specified site index, calculate the angle
# of the lat/lon array with respect to the x-axis (0-2pi counterclockwise
# from local +x-axis). 
#
# DEPENDENCIES:
# import numpy as np
# #pip install geopy
# import geopy.distance
# from geopy.distance import geodesic
#-----------------------------------------------------------------------


def calculate_beta(lead_coords, SiteIndex, step_ii, sign_conv):
    
    # define coordinates of study site
    site0LAT, site0LON = lead_coords[SiteIndex][0], lead_coords[SiteIndex][1]

    #ascending direction
    #-------------------------------------------
    # define next point
    site1LAT, site1LON = lead_coords[SiteIndex+step_ii][0],lead_coords[SiteIndex+step_ii][1]
    # find change in lat and lon between study site and final site
    dlat = geodesic((site0LAT, site0LON), (site1LAT, site0LON)).km
    dlon = geodesic((site0LAT, site0LON), (site0LAT, site1LON)).km
    # calculate angle beta from this
    beta_a = np.arctan(dlat/dlon)
    # make all angles (0<beta<2pi)
    # moving from site 0 to site 2 (along lead direction)
    if sign_conv == 'yes':
        if site1LAT > site0LAT:           # dlat>0 (quads I, II)
            if site1LON < site0LON:       # dlon<0 (quads II, III)
                beta_a = np.pi - beta_a   # quad II -> convert to angle/360
                                          # quad I  -> do nothing
        else:                             # dlat<0 (quads III, IV)
            if site1LON < site0LON:       # dlon<0 (quads II, III)
                beta_a = np.pi + beta_a   # quad III -> convert to angle/360
            else:                         # dlon>0 (quads I, IV)
                beta_a = 2*np.pi - beta_a # quad IV  -> convert to angle/360 
    # calculate arcdistance between coordinates
    ds_a = 0
    for ii in range(0, step_ii):
        ds_a=ds_a+geodesic((lead_coords[SiteIndex+ii][0], lead_coords[SiteIndex+ii][1]), (lead_coords[SiteIndex+(ii+1)][0], lead_coords[SiteIndex+(ii+1)][1])).km
    
    
    #descending direction
    #-------------------------------------------
    # define next point
    site1LAT, site1LON = lead_coords[SiteIndex-step_ii][0],lead_coords[SiteIndex-step_ii][1]
    # find change in lat and lon between study site and final site
    dlat = geodesic((site0LAT, site0LON), (site1LAT, site0LON)).km
    dlon = geodesic((site0LAT, site0LON), (site0LAT, site1LON)).km
    # calculate angle beta from this
    beta_d = np.arctan(dlat/dlon)
    # make all angles (0<beta<2pi)
    # moving from site 0 to site 2 (along lead direction)
    if sign_conv == 'yes':
        if site1LAT < site0LAT:            # dlat>0 (quads I, II)
            if site1LON > site0LON:        # dlon<0 (quads II, III)
                beta_d = np.pi - beta_d    # quad II -> convert to angle/360
                                           # quad I  -> do nothing
        else:                              # dlat<0 (quads III, IV)
            if site1LON > site0LON:        # dlon<0 (quads II, III)
                beta_d = np.pi + beta_d    # quad III -> convert to angle/360
            else:                          # dlon>0 (quads I, IV)
                beta_d = 2*np.pi - beta_d  # quad IV  -> convert to angle/360 
    # calculate arcdistance between coordinates
    ds_d = 0
    for ii in range(0, step_ii):
        ds_d=ds_d+geodesic((lead_coords[SiteIndex-ii][0], lead_coords[SiteIndex-ii][1]), (lead_coords[SiteIndex-(ii+1)][0], lead_coords[SiteIndex-(ii+1)][1])).km
 
    # approximate beta from LHS and RHS    
    Beta = (beta_d+beta_a)/2
    
    return beta_a, ds_a, beta_d, ds_d, Beta





In [ ]:
#////////////////////////////////
#  linear_interp_alongcoords ///
#//////////////////////////////
#---------------------------------------------------------------------
# given final coordinates and data values along intitial coordinates,
# linearly interpolate data along final coordinates
#---------------------------------------------------------------------
#
# INPUTS: 
#--------
# init_coords --> Nx2 array of evenly spaced coordinates ([Lat, Lon])
# init_values --> Nx1 array of values along init_coords
# final_coords --> Lx2 array of interpolated coords from init_coords 
#                 (must include all init_coords within these coordinates)
#                 (first and final values must be in init_coords)
#
# OUTPUT:  
#--------
# final_values --> Lx1 linearly interpolated data values 
#
# DEPENDENCIES:
# import numpy as np
#-----------------------------------------------------------------------


def linear_interp_alongcoords(init_coords, init_values, final_coords,suppress_prints=True):
    
    if suppress_prints==False:
        print(f'Interpolating {len(init_coords)} coords to {len(final_coords)} coords.')
   
    # run through intital coordinate list to find matches in final coordinates
    # save corresponding ii, II indices of matches
    final_ii = []
    init_II = []
    for II in range(len(init_coords)):
        for ii in range(len(final_coords)):
            # round coords to 3 floating points to stop floating point differences from 
            # interfering with matches
            if (np.round(init_coords[II],3) == np.round(final_coords[ii],3)).all():
                final_ii.append(ii)
                init_II.append(II)
       
    # run through matches and interpolate data values
    final_values = np.array([init_values[0]])
    for ll in range(len(init_II)-1):
        # grab indices of initial coordinate pair and data from first match
        II_0 = init_II[ll]
        II_1 = init_II[ll+1]
        data_0 = init_values[II_0]
        data_1 = init_values[II_1]
        # grab indices of corresponding final coordinate pair
        ii_0 = final_ii[ll]
        ii_1 = final_ii[ll+1]
        # find extra steps needed between data 0 and data 1 of current step in initial coords
        extra_steps = (ii_1-ii_0)-(II_1-II_0)
        interp_points = np.linspace(data_0, data_1, 2+extra_steps)
        final_values=np.append(final_values,interp_points[1:])
    
    return final_values




In [ ]:
#////////////////////////////
#  calculate_lead_angles ///
#//////////////////////////
#---------------------------------------------------------------------
# given lead coordinates, return lead angles, lead spacing, 
# and accumulated ditance along lead
#---------------------------------------------------------------------
#
# INPUTS: 
#--------
# lats --> Nx1 list/array of lead lats
# lons --> Nx1 list/array of lead lons
# units --> units to return degrees in 
#          (either 'deg' for degrees (defaut) or 'rad' for radians)
# suppress_prints --> bool, whether or not to suppress prints (default: True)
#
# OUTPUT:  
#--------
# angles --> forward azimuth angle between given coordinates
# spacing --> forward distance between coordinates (from corresponding point) (km)
# accumulated_distance --> accumulated distance 
#                         (up to/including corresponding point) along coordinates (km) 
#
# DEPENDENCIES:
import numpy as np
import pyproj
geodesic_path = pyproj.Geod(ellps='WGS84')
#-----------------------------------------------------------------------

# units --> units to return degrees in 
#          (either 'deg' for degrees (defaut) or 'rad' for radians)
# suppress_prints --> bool, whether or not to suppress prints (default: True)
#
# return forward azimuth angle between given coordinates
# return forward distance between coordinates (from corresponding point) (km)
# return accumulated distance (up to/including corresponding point) along coordinates (km)
def calculate_lead_angles(lats, lons, units='deg', suppress_prints = True):
    
    # lead angle and distance at point ii defines lead angle/spacing between ii and ii+1
    angles = np.array([])
    spacing = np.array([])
    accumulated_distance = np.array([0])

    
    # run through lead coordinates
    # data will be calculated for points ii=0 to ii=-1 
    # (since forward point needed to find angle)
    #-------------------------------------------------
    for ii in range(len(lats)-1):
        
        # use pyproj to find geodetic path between points
        # return forward azimuth, backward azimuth, and distance
        fwd_az,back_az,dist = geodesic_path.inv(lons[ii], lats[ii], lons[ii+1], lats[ii+1])
        
        # shift bearing angle to desired range 
        # [0,360] where 0 == east
        fwd_az = fwd_az-90
        if fwd_az < 0:
            fwd_az = np.abs(fwd_az)
        elif fwd_az>0:
            fwd_az = 360-fwd_az
            
        # save lead angle and distance to arrays  
        angles = np.append(angles,fwd_az)
        spacing = np.append(spacing, dist/1000)
        accumulated_distance = np.append(accumulated_distance, accumulated_distance[-1]+dist/1000)


    # crop accumulated_distance to not include final forward looking distance
    accumulated_distance = accumulated_distance[:-1]
    
    # convert angles to radians if specified
    if str(units) == 'rad':
        angles = angles*np.pi/180
        if suppress_prints == False:
            print('>> returning angles in radians')
    else:
        if suppress_prints == False:
            print('>> returning angles in degrees')
    
    # print average and standard deviation in coordinate spacing 
    if suppress_prints == False:
        print(f'>> coordinate spacing: {np.mean(spacing):.2f} +\- {np.std(spacing):.2f} km')
            
            
    return angles, spacing, accumulated_distance



In [1]:
#//////////////////////
#  find_mean_lead  ///
#////////////////////
#---------------------------------------------------------------------
# given list of lead coordinate lists, find mean lead position
#---------------------------------------------------------------------
# INPUTS: 
#-------
# LEADLIST --> list of lead cord txt files to import, including path
# LAT_MIN  --> starting (minimum) latitude (default: 71.5)
# LAT_MAX  --> final (maximum) latitude (default: 78)
# LAT_STEP --> step size between latitudes (default: 0.25)
# max_dist  --> (max distance (km) of lead coordinate from given latitude) (default: 10)
# lead_frac --> minimum fraction of total leads reaching latitude before stopping (default: 1/2)
# lat_col --> column of latitude data in lead coords (default: 0)
# lon_col --> column of longitude data in lead coords (default: 1)
# suppress_prints --> bool True/False, whether or not to suppress print statements as function runs
#                     (default: False)
# method --> method for finding representative lead position, either 'mean' (default) or 'median'
# return_numleads --> bool, whether or not to also return number of leads at each returned 
#                     point that were used in mean/std calculation (default:False)
# 
# OUTPUTS: 
#--------
#
# mean_lats --> latitude of mean lead
# mean_lons --> longitude of mean lead
# mean_lons_minstd  --> mean - standard deviation in longitude of mean lead
# mean_lons_plustd  --> mean + standard deviation in longitude of mean lead
#
# DEPENDENCIES:
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline
import numpy as np
import sys
from geopy.distance import geodesic
sys.path.append('/Users/mackenziejewell/Documents/GitHub/plotMODIS')
from ipynb.fs.full.LIB_general_func import import_NxM_txt
#---------------------------------------------------------------------

def find_mean_lead(LEADLIST, 
                   LAT_MIN = 71.5, LAT_MAX = 78, LAT_STEP = 0.25, 
                   max_dist = 10, lead_frac = 1/2, suppress_prints = False, 
                  lat_col=0, lon_col=1, add_spline = False, spline_order = 1, 
                   method = 'mean', return_numleads = False):

    # generate list of latitudes for leads
    mean_lats = np.arange(LAT_MIN,LAT_MAX+LAT_STEP,LAT_STEP)
    mean_lats_enoughdata = np.array([])
    mean_lons = np.array([])
    mean_lons_minstd = np.array([])
    mean_lons_plustd = np.array([])
    num_leads = np.array([])

    # run through ech latitude to search for nearest lead coords to given lat
    for mean_lat in mean_lats:
        
        nearest_lead_lons = []

        for ii in range(len(LEADLIST)):
            # import coords
            Coords = import_NxM_txt(LEADLIST[ii])
            lead_lats = Coords[:,lat_col]
            lead_lons = Coords[:,lon_col]
            nearest_lat_index = np.where(np.abs(lead_lats-mean_lat)==np.min(np.abs(lead_lats-mean_lat)))
            lead_distance = geodesic((lead_lats[nearest_lat_index],lead_lons[nearest_lat_index]), 
                                     (mean_lat, lead_lons[nearest_lat_index])).km
            
            # only save lead lon if its within MAX_DIST of current lat
            if lead_distance < max_dist:
                nearest_lead_lons.append(lead_lons[nearest_lat_index][0])  
            else:
                if suppress_prints == False:
                    print('No lead coords found within {} km of [{}] for {}'.format(max_dist,mean_lat,LEADLIST[ii]))

        # if fewer than LEAD_FRAC fraction of leads were within MAX_DIST of latitude, save nan
        # else save mean and standard deviation of longitude across all leads
        if len(nearest_lead_lons)>=len(LEADLIST)*lead_frac:
            
            mean_lats_enoughdata = np.append(mean_lats_enoughdata, mean_lat)
            
            if method == 'mean':
                mean_lons = np.append(mean_lons,np.mean(nearest_lead_lons))
            elif method == 'median':
                mean_lons = np.append(mean_lons,np.median(nearest_lead_lons))
                
            mean_lons_minstd = np.append(mean_lons_minstd,np.mean(nearest_lead_lons)-np.std(nearest_lead_lons))
            mean_lons_plustd = np.append(mean_lons_plustd,np.mean(nearest_lead_lons)+np.std(nearest_lead_lons))
            
            # grab number of leads that extend to lat
            num_leads=np.append(num_leads,len(nearest_lead_lons))

    if add_spline == True:
        SplFit = UnivariateSpline(mean_lats_enoughdata, mean_lons, s=spline_order)
        mean_lons = SplFit(mean_lats_enoughdata)
        SplFit = UnivariateSpline(mean_lats_enoughdata, mean_lons_minstd, s=spline_order)
        mean_lons_minstd = SplFit(mean_lats_enoughdata)
        SplFit = UnivariateSpline(mean_lats_enoughdata, mean_lons_plustd, s=spline_order)
        mean_lons_plustd = SplFit(mean_lats_enoughdata)
    
    if return_numleads == False:
        return mean_lats_enoughdata, mean_lons, mean_lons_minstd, mean_lons_plustd
    else:
        return mean_lats_enoughdata, mean_lons, mean_lons_minstd, mean_lons_plustd, num_leads

In [ ]:
#-----------------------------------------------------
# old version that allowed you to pick sign convention
# and directions (ascending, descending, centered) to 
# calculate beta
#-----------------------------------------------------

# def calculate_beta(lead_coords, SiteIndex, Max_ii, directions, show_plot, sign_conv):
    
#     # define coordinates of study site
#     site0LAT, site0LON = lead_coords[SiteIndex][0], lead_coords[SiteIndex][1]

#     if 'ascending' in directions:
#         # start empty arrays to fill
#         dsList_L = np.array([])
#         betaList_L = np.array([])

#         for ii in range(0, Max_ii):
#             # define coordinates of points moving away from study site
#             site1LAT, site1LON = lead_coords[SiteIndex+ii][0],lead_coords[SiteIndex+ii][1]
#             site2LAT, site2LON = lead_coords[SiteIndex+(ii+1)][0],lead_coords[SiteIndex+(ii+1)][1]
#             # find geodesic distance between two points along arc 
#             ds_L = geodesic((site1LAT, site1LON), (site2LAT, site2LON)).km
#             dsList_L = np.append(dsList_L, ds_L)
#             # find change in lat and lon between study site and final site
#             dlat = geodesic((site0LAT, site0LON), (site2LAT, site0LON)).km
#             dlon = geodesic((site0LAT, site0LON), (site0LAT, site2LON)).km
            
#             # calculate angle beta from this
#             beta = np.arctan(dlat/dlon)
            
#             # make all angles (0<beta<2pi)
#             # moving from site 0 to site 2 (along lead direction)
#             if sign_conv == 'yes':
#                 if site2LAT > site0LAT:      # dlat>0
#                     if site2LON < site0LON:  # dlon<0
#                         beta = np.pi - beta
#                 else:                        # dlat<0
#                     if site2LON < site0LON:  # dlon<0
#                         beta = np.pi + beta
#                     else:                    # dlon>0
#                         beta = 2*np.pi - beta
#             betaList_L = np.append(betaList_L, beta)
#         # find cumulative distance along arc
#         cumdsList_L = dsList_L[0]
#         for ii in range(1, len(dsList_L)): 
#             cumdsList_L = np.append(cumdsList_L, np.sum(dsList_L[0:ii+1]))
#         # make plot
#         if show_plot == 'yes': 
#             plt.plot(cumdsList_L, betaList_L, 'g-', linewidth=3, label="Left side")

#     if 'descending' in directions:
#         # start empty arrays to fill
#         dsList_R = np.array([])
#         betaList_R = np.array([])

#         for ii in range(0, Max_ii):
#              # define coordinates of points moving away from study site
#             site1LAT, site1LON = lead_coords[SiteIndex-ii][0],lead_coords[SiteIndex-ii][1]
#             site2LAT, site2LON = lead_coords[SiteIndex-(ii+1)][0],lead_coords[SiteIndex-(ii+1)][1]
#             # find geodesic distance between two points along arc 
#             ds_R = geodesic((site1LAT, site1LON), (site2LAT, site2LON)).km
#             dsList_R = np.append(dsList_R, ds_R)
#             # find change in lat and lon between study site and final site
#             dlat = geodesic((site0LAT, site0LON), (site2LAT, site0LON)).km
#             dlon = geodesic((site0LAT, site0LON), (site0LAT, site2LON)).km
#             # calculate angle beta from this
#             beta = np.arctan(dlat/dlon)
            
#             # make all angles (0<beta<2pi)
#             # moving from site 2 to site 0 (against lead direction)
#             if sign_conv == 'yes':
#                 if site0LAT > site2LAT:      # dlat>0
#                     if site0LON < site2LON:  # dlon<0
#                         beta = np.pi - beta
#                 else:                        # dlat<0
#                     if site0LON < site2LON:  # dlon<0
#                         beta = np.pi + beta
#                     else:                    # dlon>0
#                         beta = 2*np.pi - beta
#             betaList_R = np.append(betaList_R, beta)
#         # find cumulative distance along arc
#         cumdsList_R = dsList_R[0]
#         for ii in range(1, len(dsList_R)): 
#             cumdsList_R = np.append(cumdsList_R, np.sum(dsList_R[0:ii+1]))
#         # make plot
#         if show_plot == 'yes': 
#             plt.plot(cumdsList_R, betaList_R, 'b-', linewidth=3, label="Right side")

#     if 'centered' in directions:
#         # start empty arrays to fill
#         betaList_C = np.array([])
#         TotaldsList_2side = np.array([])

#         for ii in range(1, int(Max_ii/2)+1):
#             # define coordinates of points on either side of site
#             site1LAT, site1LON = lead_coords[SiteIndex-ii][0], lead_coords[SiteIndex-ii][1]
#             site2LAT, site2LON = lead_coords[SiteIndex+ii][0], lead_coords[SiteIndex+ii][1]
#             # find change in lat and lon between study site and final site
#             dlat = geodesic((site1LAT, site1LON), (site2LAT, site1LON)).km
#             dlon = geodesic((site1LAT, site1LON), (site1LAT, site2LON)).km
#             # calculate angle beta from this
#             beta = np.arctan(dlat/dlon)
#             # make all angles (0<beta<2pi)
#             if sign_conv == 'yes':
#                 if site2LAT > site1LAT:      # dlat>0
#                     if site2LON < site1LON:  # dlon<0
#                         beta = np.pi - beta
#                 else:                        # dlat<0
#                     if site2LON < site1LON:  # dlon<0
#                         beta = np.pi + beta
#                     else:                    # dlon>0
#                         beta = 2*np.pi - beta
#             betaList_C = np.append(betaList_C, beta)
#             # grab chunk of list along which calculations are being made
#             ListSect = lead_coords[SiteIndex-ii:SiteIndex+ii+1]
#             part_ds_list = np.array([])
#             for jj in range(0, len(ListSect)-1):
#                 part_ds_list = np.append(part_ds_list, geodesic((ListSect[jj, 0], ListSect[jj, 1]), (ListSect[jj+1, 0], ListSect[jj+1, 1])).km)

#             TotaldsList_2side = np.append(TotaldsList_2side, np.sum(part_ds_list))
#         # make plot
#         if show_plot == 'yes': 
#             plt.plot(TotaldsList_2side/2, betaList_C, 'r-', linewidth=3, label="Centered")
#             plt.plot(-TotaldsList_2side/2, betaList_C, 'r-', linewidth=3)

#     # approximate beta from LHS and RHS    
#     Beta = (betaList_L[-1]+betaList_R[-1])/2
    
#     # make plot
#     if show_plot == 'yes': 
#         plt.hlines(Beta, -np.max(cumdsList_R),np.max(cumdsList_R), colors='gray', linestyles='dashed', label = 'Approximate Beta')
#         plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
#         plt.ylabel('Beta (radians)')
#         plt.xlabel('Arcdistance (km) from site')
        
#     return Beta


#-------------------------------------------------------
# old version for calculating multiples points at a time
#-------------------------------------------------------

# #==========================================================================
# lead_coords = np.stack((LatArray, LonArray), axis=1)
# Max_ii = int(1)
# directions = ['ascending', 'descending']
# show_plot = 'no'

# # sign convention 'yes' if (0<beta<2pi) and 'no' if all are (0<beta<pi/2)
# sign_conv = 'yes'

# # starting index (and number of indices to stop short of end)
# St = Max_ii
# #==========================================================================

# BetaList = np.array([])
# for ii in range(0,len(lead_coords[St:-St,0])):
#     BetaList = np.append(BetaList, calculate_beta(lead_coords, ii+St, Max_ii, directions, show_plot, sign_conv))

# # cut off beginning and end of array to keep all lists consistent    
# LatArray_trim = lead_coords[St:-St,0]
# LonArray_trim = lead_coords[St:-St,1]
    
# # divide by pi to get radians
# #--------------------------------------------------------------------------
# # BetaList = BetaList/np.pi
# #--------------------------------------------------------------------------

# # make plot
# fig, axs = plt.subplots(2)
# fig.suptitle('Beta vs Site Index')
# axs[0].plot(BetaList, 'ko', MarkerSize=1)
# axs[1].scatter(LonArray_trim, LatArray_trim, c=BetaList, cmap = 'gnuplot2')
# axs[1].plot(LonArray_trim[0], LatArray_trim[0], 'ro')
# # axs[1].plot(LonArray_trim[49], LatArray_trim[49], 'ro')

